<a href="https://colab.research.google.com/github/Moojin-Bin/Hidden_Markov_Model/blob/master/HMM_v0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Application of Hidden Markov Model for Financial Markets

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from sklearn import mixture
from matplotlib import pyplot as plt

In [ ]:
def df_to_array(df):
    array = np.ravel(df).astype(np.float).reshape(-1, 1)
    return array

In [ ]:
def HMM(df, begin, n_components, log=True):

    if log == True:
        df = np.log(df[begin:].fillna(method='ffill'))
    else:
        df = df[begin:].fillna(method='ffill')

    data = df_to_array(df)

    gmm = mixture.GaussianMixture(n_components)
    gmm.fit(data)

    hidden_states = gmm.predict(data)
    hidden_states_prob = gmm.predict_proba(data)

    z = {}
    stat = {}    # Statistics = {Minimum, Maximum, Mean}
    for i in range(n_components):
        mask = hidden_states == i
        z['Regime_%d' %i] = pd.DataFrame(df.values[mask], index = df.index[mask], columns = ['Regime_%d' %i])
        z['Regime_%d' %i] = z['Regime_%d' %i].reindex(df.index, fill_value=np.nan)
        stat['Regime_%d_min' %i] = z['Regime_%d' %i].min()
        stat['Regime_%d_avg' %i] = z['Regime_%d' %i].mean()
        stat['Regime_%d_max' %i] = z['Regime_%d' %i].max()
    
    hsp = pd.DataFrame(hidden_states_prob, index=df.index, columns = z.keys())    # Hidden States Probability

    regimes = list(z.keys())
    gmm_res = pd.DataFrame([])
    for i in range(n_components):
        gmm_res = pd.concat([gmm_res, z[regimes[i]]], axis=1)

    if log == True:
        gmm_res = np.exp(gmm_res)
        stats = np.exp(pd.DataFrame.from_dict(stat, orient='index'))
    elif log == False:
        stats = pd.DataFrame.from_dict(stat, orient='index')
    
    return gmm_res, stats, hsp

In [ ]:
# Data Load
df = pdr.DataReader('VIXCLS', "fred", '19700101')